In [1]:
import pandas as pd
import re

In [2]:
table = pd.read_csv("table.csv", index_col='Орган')
points = pd.read_csv("points.csv", index_col="Точка")
pitanie = pd.read_csv("pitanie.csv", index_col='Unnamed: 0')
ke = pd.read_csv("ke.csv", index_col='Unnamed: 0')


In [4]:
table

,Стихия_усин,Инь_Ян,Стихия_сезон,Gui,Xue,Luo,Jing_Jin,Yuan
Орган,,,,,,,,
Gb,дерево,Yang,вода,Gv23,Gb36,Gb37,Gb44,Gb40
Si,огонь,Yang,земля,HG8(HG7),Si6,Si7,Si1,Si4
Th,огонь,Yang,вода,Sp1,Th7,Th5,Th1,Th4
St,земля,Yang,земля,St6,St34,St40,St45,St42
Co,металл,Yang,дерево,Cv24,Co7,Co6,Co1,Co4
Bl,вода,Yang,металл,HG7(HG5),Bl63,Bl58,Bl67,Bl64
Liv,дерево,Yin,земля,Gv16,Liv6,Liv5,Liv1,Liv3
Ht,огонь,Yin,огонь,Lu11,Ht6,Ht5,Ht9,Ht7
Hg,огонь,Yin,земля,Bl62,Hg4,Hg6,Hg9,Hg7


In [3]:
{"t":1, "f":2, "e":3, "m":4, "w":5}


def kachestvo(chan):
    '''
    Принимает на вход два канала и возвращает их общие качества
    '''
    dict_of_colors = {'Liv':[4,3,1,1], 'Gb':[3,5,1,2], 'Ht':[1,2,2,2], 'Si':[5,3,2,5], 'Hg':[3,2,1], 
                    'Th':[5,2,2], 'Sp':[3,2,3,3], 'St':[2,3,3,4], 'Lu':[5,1,4,3], 'Co':[4,1,4,4], 'Kid':[2,4,5,2], 'Bl':[1,4,5,5]}
    return str(set(dict_of_colors[chan[0]]) & set(dict_of_colors[chan[1]])).strip('{}')


In [157]:
points = pd.read_csv("points.csv", index_col="Точка")#.drop('Unnamed: 0', axis=1)

In [159]:
points.loc['Bl58', ['Название',	'Значение',	'Локализация']] = \
    ['«Фэй ян»', 
    '«Фэй ян» — «взлететь, взметнуться». Коллатераль ножного тай-ян канала мочевого пузыря отходит («взлетает») от точки Фэй-ян Bl58 и соединяется с ножным шао-инь каналом почек.', 
    'на 7 цуней прямо кверху от точки Кунь-лунь Bl60 (в углублении между латеральной лодыжкой и пяточным сухожилием)']
points.loc['Bl58',:]

Название                                                «Фэй ян»
Значение       «Фэй ян» — «взлететь, взметнуться». Коллатерал...
Локализация    на 7 цуней прямо кверху от точки Кунь-лунь Bl6...
Name: Bl58, dtype: object

In [160]:
points.to_csv("points.csv")

In [125]:
from os import listdir
from os.path import isfile, join

mypath = './patients/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['all_patients.csv',
 'patients.csv',
 'Богданов М.А..csv',
 'Богданов Максим Александрович.csv',
 'Бочкова Людмила Ованесовна.csv',
 'Денисова Светлана Александровна.csv',
 'Ермакова Людмила Ивановна.csv',
 'Карасёва Наталья Борисовна.csv',
 'Круглова Елена Викторовна.csv',
 'Миронова Наталья Николаевна.csv',
 'Носкова Нина Павловна.csv',
 'Носкова Ольга Геннадьевна.csv',
 'Шапошников Сергей Анатольевич.csv',
 'Шевченко Надежда Николаевна.csv']

In [243]:
from datetime import datetime
import csv
# new_save = pd.DataFrame(
#     # index=[datetime.now().date()],
#     data=[[datetime.now().date(), '', '', '', '', '', '', '']],
#     columns=['Дата', 'ФИО', 'дата рождения', 'жалобы', 'метод лечения', 'дата события', 'лечение', 'комментарии']
# )
new_save = [datetime.now().date(), 'd', 'sd', 'sd', '', 'sd', '', 'sd']

In [254]:

with open('patients/all_patients.csv', 'rb') as f:
    encoding.detect(f.read())

AttributeError: '_io.BufferedReader' object has no attribute 'detect'

In [260]:
# save_all = pd.read_csv("patients/patients.csv", index_col='Дата')   chardet.detect(f.read())
# pd.concat([patients[:2], new_save], axis=0)#.drop_duplicates()

with open('patients/all_patients.csv', 'a') as file:
    writer = csv.writer(file)
    writer.writerow(new_save) 
file.close()

# with open('eggs.csv', 'w', newline='') as csvfile:
#     spamwriter = csv.writer(csvfile, delimiter=' ',
#                             quotechar='|', quoting=csv.QUOTE_MINIMAL)
#     spamwriter.writerow(['Spam'] * 5 + ['Baked Beans'])
#     spamwriter.writerow(['Spam', 'Lovely Spam', 'Wonderful Spam'])

In [265]:
patients = pd.read_csv("patients/patients.csv")#.drop('Unnamed: 0', axis=1)#,index_col="Дата", parse_dates=True
# patients = pd.read_csv(f"patients/{onlyfiles[2]}")
# for file in onlyfiles[3:]:
#     patient = pd.read_csv(f"patients/{file}")
#     patients = pd.concat([patients, patient], axis=0)

# # patients['Unnamed: 0'] = patients['Unnamed: 0'].str.replace("/", ".")
# patients['Unnamed: 0'] = pd.to_datetime(patients['Unnamed: 0'], format="mixed", dayfirst=True)
# patients = patients.sort_values("Unnamed: 0")
# patients = patients.rename(columns={"Unnamed: 0":"Дата"})
# patients = patients.reset_index(drop=True)
# # patients
# patients.drop('Unnamed: 0.1', axis=1, inplace=True)

patients[-2:]

,Дата,ФИО,дата рождения,жалобы,метод лечения,застой,недостаток,блок,Gui,противоток,xue,пат.рост,жар,холод,сырость,сухость,лечение,комментарии,дата события
15,2025-01-18,Стругин Константин Геннадьевич,27.05.1968,Боли в спине начинаются после физической нагру...,Питание и Ке,"['Co', 'Bl']","['Liv', 'Kid']",[],"['Ht', 'Bl', 'Lu', 'St']",[],[],[],[],[],[],[],"Lu11, HG7(HG5), Gv26, St6 (d*-s*), Lu9 (s')",При хорошем эффекте на основных каналах в план...,NaN
16,2025-01-18,Бочкарева Альбина Анатольевна,08.05.1973,Боли в правой ноге по латеральной поверхности ...,Лунные дворцы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bl58 (d'-s'),NaN,08.05.2016


In [216]:
# from datetime import datetime
# new_save = pd.DataFrame(
#     index=[datetime.now().date()],
#     data=0,
#     columns=['ФИО', 'дата рождения', 'жалобы', 'метод лечения', 'дата события', 'лечение', 'комментарии']
# )
# # save_all = pd.read_csv("patients/patients.csv", index_col='Дата')
# pd.concat([patients[:2], new_save], axis=0)#.drop_duplicates()

In [241]:

patients.to_csv("patients/patients.csv", index=False)
# name = 'Стругин Константин Геннадьевич'
# patients[patients['ФИО']==name]#[-1:].dropna(axis=1)

In [249]:
'None, ' * 11

'None, None, None, None, None, None, None, None, None, None, None, '